In [77]:
import pandas as pd
from scipy import stats
from sklearn.metrics import roc_auc_score
from scipy.signal import correlate
import numpy as np
import matplotlib.pyplot as plt

| Metric                      | Thresholds (Rule of Thumb)                                                                                  | Meaning                                                                     |
| --------------------------- | ----------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------- |
| **p-value**                 | < 0.05 → significant  <br> ≥ 0.05 → not significant                                                         | Is the difference between fault vs non-fault **real** or just random noise? |
| **Cohen’s d** (effect size) | 0.2 = small  <br> 0.5 = medium  <br> 0.8+ = large                                                           | How **big** the difference is between the two groups.                       |
| **AUC**                     | 0.5 = random (useless) <br> 0.6–0.7 = weak  <br> 0.7–0.8 = fair  <br> 0.8–0.9 = good  <br> >0.9 = excellent | How well the feature can **separate fault vs non-fault**.                   |


📌 How to Decide if a Feature is Useful

Check p-value

If ≥ 0.05 → ignore the feature (no significant difference).

If < 0.05 → continue.

Check effect size (Cohen’s d)

d < 0.2 → trivial difference → probably useless.

d ≥ 0.2 → meaningful → keep candidate.

Check discrimination (AUC)

AUC < 0.6 → poor, feature is weak.

AUC 0.6–0.7 → weak but may help when combined.

AUC ≥ 0.7 → strong candidate feature.

In [78]:
df_joint0_stop = pd.read_csv('Dataset\joint_0_features (1) stop.csv')
df_joint0_stop.head()

,Num,Timestamp,Current_J0,Temperature_T0,Speed_J0,Tool_current,cycle,Robot_ProtectiveStop
0,1,2022-10-26T08:17:21.847Z,0.109628,27.875,2.955651e-01,0.082732,1,0.0
1,2,2022-10-26T08:17:22.852Z,0.595605,27.875,-7.390000e-30,0.505895,1,0.0
2,3,2022-10-26T08:17:23.857Z,-0.229474,27.875,1.369386e-01,0.079420,1,0.0
3,4,2022-10-26T08:17:24.863Z,0.065053,27.875,-9.030032e-02,0.083325,1,0.0
4,5,2022-10-26T08:17:25.877Z,0.884140,27.875,1.268088e-01,0.086379,1,0.0


In [79]:
df_joint0_grip = pd.read_csv('Dataset\joint_0_features (1)_grip.csv')
df_joint0_grip.head()

,Num,Timestamp,Current_J0,Temperature_T0,Speed_J0,Tool_current,cycle,grip_lost
0,1,2022-10-26T08:17:21.847Z,0.109628,27.875,2.955651e-01,0.082732,1,False
1,2,2022-10-26T08:17:22.852Z,0.595605,27.875,-7.390000e-30,0.505895,1,False
2,3,2022-10-26T08:17:23.857Z,-0.229474,27.875,1.369386e-01,0.079420,1,False
3,4,2022-10-26T08:17:24.863Z,0.065053,27.875,-9.030032e-02,0.083325,1,False
4,5,2022-10-26T08:17:25.877Z,0.884140,27.875,1.268088e-01,0.086379,1,False


In [80]:
def convert_to_timestamp(df):
    df["Timestamp"] = df["Timestamp"].str.replace('"', '', regex=False)

    df["Timestamp"] = pd.to_datetime(df["Timestamp"], utc=True, errors="coerce")

In [81]:

def remove_strip(df):
        df.columns = df.columns.str.strip()
        return df

df_joint0_stop=remove_strip(df_joint0_stop)
df_joint0_grip=remove_strip(df_joint0_grip)        


In [82]:
def fill_null_with_fandb(df):
    clean_data = df.groupby('cycle').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
    return clean_data

In [83]:
df_joint0_stop_clean=fill_null_with_fandb(df_joint0_stop)
df_joint0_grip_clean=fill_null_with_fandb(df_joint0_grip)
df_joint0_grip_clean.isnull().sum()

C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_data = df.groupby('cycle').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clean_data = df.groupby('cycle').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

Num               0
Timestamp         0
Current_J0        0
Temperature_T0    0
Speed_J0          0
Tool_current      0
cycle             0
grip_lost         0
dtype: int64

In [84]:
def add_robot_features(df, dt=1.0, window=5):
    df = df.copy()

    curr_cols = [c for c in df.columns if c.startswith('Current_J')]
    spd_cols  = [c for c in df.columns if c.startswith('Speed_J')]
    tmp_cols  = [c for c in df.columns if c.startswith('Temperature')]

    # Loop joints (J0..J5)
    for j in range(6):
        c, s = f'Current_J{j}', f'Speed_J{j}'
        if c in df and s in df:
            # Absolute values
            df[f'abs_{c}'] = df[c].abs()
            df[f'abs_{s}'] = df[s].abs()

            # Sudden change (derivative)
            df[f'sudden{j}'] = df[s].diff() / dt

            # Power (approx)
            df[f'power_J{j}'] = df[c].abs() * df[s].abs()

            # Rolling means
            df[f'speed{j}_roll_mean'] = df[s].rolling(window).mean()
            df[f'current{j}_roll_mean'] = df[c].rolling(window).mean()

    # --- Extra engineered features (global, not per joint) ---
    if "Tool_current" in df and "Current_J5" in df:
        df["ratio_tool_current"] = df["Tool_current"] / (df["Current_J5"] + 1e-6)
    if "Temperature_J5" in df and "Current_J5" in df:
        df["temp_per_current"] = df["Temperature_J5"] / (df["Current_J5"] + 1e-6)
    if "Speed_J5" in df and "Current_J5" in df:
        df["speed_per_current"] = df["Speed_J5"] / (df["Current_J5"] + 1e-6)
    if {"Current_J5","Temperature_J5","Speed_J5"} <= set(df.columns):
        df["interaction_all"] = df["Current_J5"] * df["Temperature_J5"] * df["Speed_J5"]
    if {"Current_J5","Tool_current","Speed_J5"} <= set(df.columns):
        df["load_efficiency"] = (df["Current_J5"] + df["Tool_current"]) / (df["Speed_J5"] + 1e-6)

    return df


In [85]:


def univariate_analysis(df, features, target):
    y = df[target].astype(int).values
    results = []
    for f in features:
        x = df[f].values
     
        x0, x1 = x[y==0], x[y==1]
        
        t, p = stats.ttest_ind(x0, x1, equal_var=False, nan_policy="omit")
        
        s0, s1 = x0.std(ddof=1), x1.std(ddof=1)
        sp = np.sqrt(((len(x0)-1)*s0**2 + (len(x1)-1)*s1**2) / (len(x0)+len(x1)-2))
        d = (x1.mean() - x0.mean()) / sp if sp > 0 else np.nan
        
        auc = roc_auc_score(y, x) if len(np.unique(y)) == 2 else np.nan
        results.append({
            "feature": f,
            "mean_no_fault": np.mean(x0),
            "mean_fault": np.mean(x1),
            "cohens_d": d,
            "t_stat": t,
            "p_value": p,
            "AUC": auc
        })
    return pd.DataFrame(results).sort_values("p_value")






In [86]:
df_joint1=pd.read_csv('Dataset/joint_1_features (1).csv')
df_j2=pd.read_csv('Dataset/joint_2_features (1).csv')
df_j3=pd.read_csv("Dataset/joint_3_features (2).csv")
df_j4=pd.read_csv("Dataset/joint_4_features (1).csv")
df_j5=pd.read_csv("Dataset/joint_5_features (1) (1).csv")


In [87]:
convert_to_timestamp(df_joint1)
convert_to_timestamp(df_j2)
convert_to_timestamp(df_j3)
convert_to_timestamp(df_j4)
convert_to_timestamp(df_j5)

In [88]:
remove_strip(df_joint1)
remove_strip(df_j2)
remove_strip(df_j3)
remove_strip(df_j4)
remove_strip(df_j5)

,Num,Timestamp,Current_J5,Temperature_J5,Speed_J5,Tool_current,cycle,Robot_ProtectiveStop,grip_lost
0,1,2022-10-26 08:17:21.847000+00:00,-0.152622,32.0000,-0.152962,0.082732,1,0.0,False
1,2,2022-10-26 08:17:22.852000+00:00,-0.260764,32.0000,0.000417,0.505895,1,0.0,False
2,3,2022-10-26 08:17:23.857000+00:00,0.039071,32.0625,-0.496856,0.079420,1,0.0,False
3,4,2022-10-26 08:17:24.863000+00:00,0.153903,32.0000,0.425559,0.083325,1,0.0,False
4,5,2022-10-26 08:17:25.877000+00:00,0.178998,32.0000,0.180989,0.086379,1,0.0,False
...,...,...,...,...,...,...,...,...,...
7404,7405,2022-10-26 15:36:02.555000+00:00,0.023025,44.5625,0.000000,0.080013,264,0.0,False
7405,7406,2022-10-26 15:36:03.562000+00:00,0.047794,44.5000,0.000000,0.078740,264,0.0,False
7406,7407,2022-10-26 15:36:04.571000+00:00,0.028567,44.5000,0.000000,0.091471,264,0.0,False
7407,7408,2022-10-26 15:36:05.572000+00:00,0.049641,44.5625,0.000000,0.090026,264,0.0,False


In [89]:
df_joint1=fill_null_with_fandb(df_joint1)
df_joint2=fill_null_with_fandb(df_j2)
df_joint3=fill_null_with_fandb(df_j3)
df_joint4=fill_null_with_fandb(df_j4)
df_joint5=fill_null_with_fandb(df_j5)

C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_data = df.groupby('cycle').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clean_data = df.groupby('cycle').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\428220627.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

In [93]:
df_joint0_stop_clean=add_robot_features(df_joint0_stop_clean)
df_joint0_grip_clean=add_robot_features(df_joint0_grip_clean)
df_joint1=add_robot_features(df_joint1)
df_joint2=add_robot_features(df_joint2)
df_joint3=add_robot_features(df_joint3)
df_joint4=add_robot_features(df_joint4)
df_joint5=add_robot_features(df_joint5)   

In [94]:
df_joint0_stop_clean.isnull().sum()

Num                     0
Timestamp               0
Current_J0              0
Temperature_T0          0
Speed_J0                0
Tool_current            0
cycle                   0
Robot_ProtectiveStop    0
abs_Current_J0          0
abs_Speed_J0            0
sudden0                 1
power_J0                0
speed0_roll_mean        4
current0_roll_mean      4
dtype: int64

In [95]:
df_joint0_stop_clean["sudden0"].fillna(df_joint0_stop_clean["sudden0"].mean(), inplace=True)
df_joint0_grip_clean["sudden0"].fillna(df_joint0_grip_clean["sudden0"].mean(), inplace=True)
df_joint1["sudden1"].fillna(df_joint1["sudden1"].mean(), inplace=True)
df_joint2["sudden2"].fillna(df_joint2["sudden2"].mean(), inplace=True)
df_joint3["sudden3"].fillna(df_joint3["sudden3"].mean(), inplace=True)
df_joint4["sudden4"].fillna(df_joint4["sudden4"].mean(), inplace=True)
df_joint5["sudden5"].fillna(df_joint5["sudden5"].mean(), inplace=True)

C:\Users\youss\AppData\Local\Temp\ipykernel_8088\3349680284.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_joint0_stop_clean["sudden0"].fillna(df_joint0_stop_clean["sudden0"].mean(), inplace=True)
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\3349680284.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [98]:
df_joint0_stop_clean["speed0_roll_mean"].fillna(df_joint0_stop_clean["speed0_roll_mean"].mean(), inplace=True)
df_joint0_stop_clean["current0_roll_mean"].fillna(df_joint0_stop_clean["current0_roll_mean"].mean(), inplace=True)
df_joint0_grip_clean["speed0_roll_mean"].fillna(df_joint0_grip_clean["speed0_roll_mean"].mean(), inplace=True)
df_joint0_grip_clean["current0_roll_mean"].fillna(df_joint0_grip_clean["current0_roll_mean"].mean(), inplace=True)
df_joint1["speed1_roll_mean"].fillna(df_joint1["speed1_roll_mean"].mean(), inplace=True)
df_joint1["current1_roll_mean"].fillna(df_joint1["current1_roll_mean"].mean(), inplace=True)
df_joint2["speed2_roll_mean"].fillna(df_joint2["speed2_roll_mean"].mean(), inplace=True)
df_joint2["current2_roll_mean"].fillna(df_joint2["current2_roll_mean"].mean(), inplace=True)
df_joint3["speed3_roll_mean"].fillna(df_joint3["speed3_roll_mean"].mean(), inplace=True)
df_joint3["current3_roll_mean"].fillna(df_joint3["current3_roll_mean"].mean(), inplace=True)
df_joint4["speed4_roll_mean"].fillna(df_joint4["speed4_roll_mean"].mean(), inplace=True)
df_joint4["current4_roll_mean"].fillna(df_joint4["current4_roll_mean"].mean(), inplace=True)

df_joint5["speed5_roll_mean"].fillna(df_joint5["speed5_roll_mean"].mean(), inplace=True)
df_joint5["current5_roll_mean"].fillna(df_joint5["current5_roll_mean"].mean(), inplace=True)


C:\Users\youss\AppData\Local\Temp\ipykernel_8088\1880026115.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_joint0_stop_clean["speed0_roll_mean"].fillna(df_joint0_stop_clean["speed0_roll_mean"].mean(), inplace=True)
C:\Users\youss\AppData\Local\Temp\ipykernel_8088\1880026115.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

In [103]:
df_joint0_stop_clean.head()

Num                 Timestamp  Current_J0  Temperature_T0  \
cycle                                                                
1     0    1  2022-10-26T08:17:21.847Z    0.109628          27.875   
      1    2  2022-10-26T08:17:22.852Z    0.595605          27.875   
      2    3  2022-10-26T08:17:23.857Z   -0.229474          27.875   
      3    4  2022-10-26T08:17:24.863Z    0.065053          27.875   
      4    5  2022-10-26T08:17:25.877Z    0.884140          27.875   

             Speed_J0  Tool_current  cycle  Robot_ProtectiveStop  \
cycle                                                              
1     0  2.955651e-01      0.082732      1                   0.0   
      1 -7.390000e-30      0.505895      1                   0.0   
      2  1.369386e-01      0.079420      1                   0.0   
      3 -9.030032e-02      0.083325      1                   0.0   
      4  1.268088e-01      0.086379      1                   0.0   

         abs_Current_J0  abs_Speed_J0   sudden0      power_J0  \
cycle                                                           
1     0        0.109628  2.955651e-01 -0.000040  3.240209e-02   
      1        0.595605  7.390000e-30 -0.295565  4.401522e-30   
      2        0.229474  1.369386e-01  0.136939  3.142382e-02   
      3        0.065053  9.030032e-02 -0.227239  5.874323e-03   
      4        0.884140  1.268088e-01  0.217109  1.121168e-01   

         speed0_roll_mean  current0_roll_mean  
cycle                                          
1     0          0.001815           -0.048254  
      1          0.001815           -0.048254  
      2          0.001815           -0.048254  
      3          0.001815           -0.048254  
      4          0.093802            0.284990

In [104]:

features = ["Current_J0","Temperature_T0","Speed_J0","Tool_current","abs_Current_J0","abs_Speed_J0","sudden0","power_J0","speed0_roll_mean","current0_roll_mean"]
uni_stop = univariate_analysis(df_joint0_stop_clean, features, "Robot_ProtectiveStop")
uni_stop

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
5,abs_Speed_J0,0.073784,0.009527,-0.493558,23.531649,9.237980e-88,0.334439
7,power_J0,0.025056,0.004641,-0.373816,13.664739,1.636070e-35,0.337293
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
1,Temperature_T0,34.884283,35.730063,0.307080,-6.138063,2.530407e-09,0.593254
4,abs_Current_J0,0.377315,0.235823,-0.189761,4.215518,3.242020e-05,0.364578
2,Speed_J0,0.002116,-0.004915,-0.047201,2.417429,1.587951e-02,0.467948
0,Current_J0,-0.051095,0.025440,0.091966,-2.087744,3.760191e-02,0.565948
6,sudden0,0.000085,-0.003243,-0.015676,0.428180,6.687844e-01,0.479029
9,current0_roll_mean,-0.047885,-0.057699,-0.027832,0.409708,6.823170e-01,0.513440
8,speed0_roll_mean,0.001839,0.001191,-0.011103,0.166941,8.675307e-01,0.501023


In [105]:

uni_grip = univariate_analysis(df_joint0_grip_clean, features, "grip_lost")
uni_grip

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
1,Temperature_T0,34.960630,33.603909,-0.493646,5.930738,9.905253e-09,0.402044
7,power_J0,0.024599,0.015108,-0.173424,5.413614,1.198579e-07,0.546177
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
4,abs_Current_J0,0.375206,0.277060,-0.131578,2.776960,5.863169e-03,0.476163
2,Speed_J0,0.001235,0.020005,0.126033,-2.490364,1.336036e-02,0.566018
9,current0_roll_mean,-0.050079,0.005567,0.157863,-2.447415,1.505297e-02,0.552507
8,speed0_roll_mean,0.001538,0.009979,0.144492,-2.176715,3.040865e-02,0.544138
6,sudden0,-0.000570,0.015601,0.076167,-1.640820,1.019685e-01,0.542892
0,Current_J0,-0.050048,0.005904,0.067228,-1.405197,1.610915e-01,0.514501
5,abs_Speed_J0,0.071277,0.073923,0.020232,-0.446117,6.558588e-01,0.574373


In [107]:
features1= ["Current_J1","Temperature_J1","Speed_J1","Tool_current","abs_Current_J1","abs_Speed_J1","sudden1","power_J1","speed1_roll_mean","current1_roll_mean"]
uni_j1 = univariate_analysis(df_joint1, features1, "Robot_ProtectiveStop")
uni_j1


,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
9,current1_roll_mean,-2.282774,-2.663735,-0.863674,13.339192,3.896771e-32,0.260390
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
4,abs_Current_J1,2.289062,2.757940,0.611624,-8.247045,5.570898e-15,0.698588
0,Current_J1,-2.279789,-2.740640,-0.579671,7.701712,2.099780e-13,0.301251
1,Temperature_J1,37.633126,38.623880,0.305967,-6.131565,2.622704e-09,0.588581
8,speed1_roll_mean,-0.000177,0.006843,0.381007,-4.353740,1.861785e-05,0.565046
2,Speed_J1,-0.000439,0.013557,0.284206,-2.722837,6.871930e-03,0.522557
7,power_J1,0.053733,0.070446,0.134505,-1.332034,1.839124e-01,0.480566
5,abs_Speed_J1,0.020453,0.025616,0.115290,-1.039445,2.994826e-01,0.474217
6,sudden1,-0.000065,0.001650,0.023105,-0.197453,8.436151e-01,0.517130


In [108]:
uni_grip1=univariate_analysis(df_joint1, features1, "grip_lost")
uni_grip1

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
9,current1_roll_mean,-2.307089,-2.003130,0.685016,-18.481198,1.651224e-51,0.726331
7,power_J1,0.055579,0.018489,-0.298831,13.423900,4.354066e-35,0.519826
5,abs_Speed_J1,0.021038,0.009139,-0.265983,9.544974,2.050629e-19,0.525263
4,abs_Current_J1,2.316533,2.017274,-0.388675,8.817925,1.240856e-16,0.362583
0,Current_J1,-2.306634,-2.017274,0.362523,-8.502680,1.060143e-15,0.637393
1,Temperature_J1,37.723538,36.104424,-0.501158,6.047101,5.294028e-09,0.404347
8,speed1_roll_mean,0.000206,-0.003387,-0.194585,4.690176,4.224130e-06,0.421088
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
2,Speed_J1,0.000261,-0.005029,-0.107293,3.882864,1.224488e-04,0.404765
6,sudden1,0.000210,-0.006177,-0.086051,2.356836,1.906657e-02,0.475092


In [110]:
features2=["Current_J2","Temperature_J2","Speed_J2","Tool_current","abs_Current_J2","abs_Speed_J2","sudden2","power_J2","speed2_roll_mean","current2_roll_mean"]
uni_j2=univariate_analysis(df_joint2, features2, "Robot_ProtectiveStop")
uni_j2

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
5,abs_Speed_J2,0.242328,0.012653,-0.332080,22.875729,4.426332e-105,0.385802
9,current2_roll_mean,-1.188263,-1.489898,-0.990840,13.455804,1.857133e-32,0.227226
7,power_J2,0.300252,0.032094,-0.283174,12.219410,2.337507e-30,0.395332
4,abs_Current_J2,1.216821,1.624308,0.760526,-12.324397,1.644947e-28,0.735775
8,speed2_roll_mean,0.011256,-0.158124,-0.603363,9.852352,5.045736e-20,0.349092
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
0,Current_J2,-1.186468,-1.536902,-0.577907,7.651977,2.902478e-13,0.274792
1,Temperature_J2,38.037202,39.037410,0.302871,-6.087680,3.350346e-09,0.589816
6,sudden2,-0.007862,0.200909,0.202106,-4.668770,4.435042e-06,0.531417
2,Speed_J2,0.005259,-0.008568,-0.018906,1.322260,1.861931e-01,0.525518


In [111]:
uni_grip2=univariate_analysis(df_joint2, features2, "grip_lost")
uni_grip2

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
9,current2_roll_mean,-1.204342,-1.060436,0.466120,-11.871467,8.160480e-27,0.680326
4,abs_Current_J2,1.241158,0.966972,-0.508528,11.248638,1.837906e-24,0.319742
0,Current_J2,-1.207555,-0.966972,0.395345,-9.776004,1.068605e-19,0.673261
1,Temperature_J2,38.130181,36.443673,-0.511956,6.143732,3.129258e-09,0.402644
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
7,power_J2,0.294750,0.154622,-0.147811,3.825989,1.589214e-04,0.504806
2,Speed_J2,-0.000079,0.146782,0.200940,-3.720530,2.423324e-04,0.532578
8,speed2_roll_mean,0.003021,0.059620,0.200426,-2.941440,3.564703e-03,0.576733
5,abs_Speed_J2,0.235541,0.178752,-0.081954,1.462688,1.447366e-01,0.516476
6,sudden2,0.002414,-0.071201,-0.071218,0.978522,3.287431e-01,0.484348


In [112]:
features3=["Current_J3","Temperature_J3","Speed_J3","Tool_current","abs_Current_J3","abs_Speed_J3","sudden3","power_J3","speed3_roll_mean","current3_roll_mean"]
uni_j3=univariate_analysis(df_joint3, features3, "Robot_ProtectiveStop")
uni_j3

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
5,abs_Speed_J3,0.212054,0.027808,-0.624787,27.157006,4.345701e-102,0.352676
7,power_J3,0.153044,0.023735,-0.468830,19.506943,3.027509e-63,0.366516
4,abs_Current_J3,0.667884,0.952615,0.695053,-11.412669,2.667147e-25,0.730963
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
9,current3_roll_mean,-0.600534,-0.734479,-0.539398,7.076337,1.104784e-11,0.365218
1,Temperature_J3,40.912009,41.841622,0.292968,-5.914450,8.712821e-09,0.576520
8,speed3_roll_mean,-0.005766,0.021479,0.224946,-4.576375,6.824219e-06,0.558550
0,Current_J3,-0.598820,-0.777877,-0.348873,4.314412,2.197372e-05,0.317043
2,Speed_J3,-0.005795,0.022187,0.077542,-3.831301,1.396372e-04,0.509185
6,sudden3,-0.000409,0.010931,0.021390,-0.553419,5.803401e-01,0.483868


In [113]:
uni_j3_grip=univariate_analysis(df_joint3, features3, "grip_lost")
uni_j3_grip

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
1,Temperature_J3,41.002869,39.299897,-0.538319,6.335039,1.087738e-09,0.392211
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
0,Current_J3,-0.603463,-0.667481,-0.124489,3.519958,4.966989e-04,0.416511
2,Speed_J3,-0.002425,-0.073057,-0.195826,2.835093,4.945851e-03,0.390593
5,abs_Speed_J3,0.203704,0.246745,0.144981,-2.192538,2.923338e-02,0.565661
6,sudden3,0.001851,-0.054048,-0.105456,1.510398,1.321733e-01,0.491148
9,current3_roll_mean,-0.605194,-0.616912,-0.046943,1.440785,1.506139e-01,0.435170
8,speed3_roll_mean,-0.004468,-0.012782,-0.068593,1.232108,2.189986e-01,0.470412
4,abs_Current_J3,0.678919,0.669386,-0.023071,0.544329,5.866398e-01,0.558109
7,power_J3,0.148065,0.151409,0.012077,-0.262733,7.929508e-01,0.578433


In [114]:
features4=["Current_J4","Temperature_J4","Speed_J4","Tool_current","abs_Current_J4","abs_Speed_J4","sudden4","power_J4","speed4_roll_mean","current4_roll_mean"]
uni_j4=univariate_analysis(df_joint4, features4, "Robot_ProtectiveStop")
uni_j4

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
5,abs_Speed_J4,0.015187,0.001500,-0.386009,19.229266,5.519968e-66,0.304044
7,power_J4,0.005335,0.001151,-0.261117,11.831743,5.484365e-29,0.315943
8,speed4_roll_mean,-0.001236,0.008334,0.547995,-10.592035,1.404617e-22,0.668732
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
1,Temperature_J4,42.575868,43.650762,0.293163,-5.920029,8.450589e-09,0.576913
6,sudden4,0.000310,-0.007902,-0.152111,3.404421,7.467793e-04,0.467629
0,Current_J4,-0.025086,0.032600,0.091643,-1.336861,1.822976e-01,0.590790
2,Speed_J4,-0.000909,-0.000035,0.022724,-1.181462,2.377977e-01,0.478270
4,abs_Current_J4,0.303311,0.346034,0.077511,-1.132862,2.581915e-01,0.497636
9,current4_roll_mean,-0.023223,-0.011248,0.042050,-0.468407,6.398483e-01,0.507232


In [115]:
uni_grip4=univariate_analysis(df_joint4, features4, "grip_lost")
uni_grip4

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
1,Temperature_J4,42.681334,40.699846,-0.542094,6.356244,9.663760e-10,0.364958
2,Speed_J4,-0.000430,-0.014029,-0.354347,6.227138,1.857816e-09,0.432294
4,abs_Current_J4,0.309278,0.176402,-0.241268,6.153832,2.471276e-09,0.503494
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
9,current4_roll_mean,-0.020892,-0.078194,-0.201341,4.486533,1.057799e-05,0.402192
6,sudden4,0.000329,-0.009672,-0.185267,3.759536,2.082543e-04,0.458132
5,abs_Speed_J4,0.014458,0.020989,0.183792,-3.380453,8.319059e-04,0.619364
0,Current_J4,-0.020725,-0.087472,-0.106041,2.779975,5.782823e-03,0.387992
8,speed4_roll_mean,-0.000827,-0.002316,-0.084828,1.420774,1.565706e-01,0.453295
7,power_J4,0.005206,0.004341,-0.053949,1.417393,1.574138e-01,0.589722


In [118]:
features5=["Current_J5","Temperature_J5","Speed_J5","Tool_current","abs_Current_J5","abs_Speed_J5","sudden5","power_J5","speed5_roll_mean","current5_roll_mean"]
uni_j5=univariate_analysis(df_joint5, features5, "Robot_ProtectiveStop")
uni_j5

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
5,abs_Speed_J5,0.170690,0.012567,-0.592789,37.637339,3.253891e-219,0.312244
7,power_J5,0.025144,0.001397,-0.515755,34.614775,2.861154e-207,0.306313
4,abs_Current_J5,0.105448,0.051321,-0.703946,19.414696,1.604618e-57,0.290386
3,Tool_current,0.110511,0.088449,-0.279074,8.781327,5.679781e-17,0.419020
1,Temperature_J5,41.861886,42.928315,0.290649,-5.867422,1.125700e-08,0.547004
0,Current_J5,0.001656,-0.021978,-0.182921,5.720646,2.166046e-08,0.404955
9,current5_roll_mean,0.000250,0.014434,0.219828,-4.278470,2.509320e-05,0.561403
8,speed5_roll_mean,0.003504,-0.014025,-0.165373,3.579742,3.973930e-04,0.446582
2,Speed_J5,0.003136,-0.005504,-0.027435,1.834053,6.679414e-02,0.481528
6,sudden5,0.000312,-0.007438,-0.016217,0.464337,6.426894e-01,0.546164


In [119]:
uni5_grip=univariate_analysis(df_joint5, features5, "grip_lost")
uni5_grip

,feature,mean_no_fault,mean_fault,cohens_d,t_stat,p_value,AUC
1,Temperature_J5,41.965506,40.030607,-0.528870,6.338463,1.064874e-09,0.353946
4,abs_Current_J5,0.102621,0.126663,0.310380,-4.799189,2.694349e-06,0.593194
3,Tool_current,0.110131,0.096362,-0.174017,4.621603,5.669026e-06,0.495374
0,Current_J5,0.001897,-0.032591,-0.267069,3.666675,2.989937e-04,0.426762
9,current5_roll_mean,0.001189,-0.011159,-0.191327,2.626497,9.148680e-03,0.444873
7,power_J5,0.023983,0.032115,0.175844,-2.366514,1.870566e-02,0.578530
2,Speed_J5,0.001246,0.048957,0.151542,-2.084053,3.815030e-02,0.653928
5,abs_Speed_J5,0.163782,0.192879,0.108415,-1.499487,1.349828e-01,0.562683
6,sudden5,-0.000964,0.029047,0.062799,-0.803432,4.224784e-01,0.508522
8,speed5_roll_mean,0.002818,0.003619,0.007551,-0.106111,9.155776e-01,0.520493
